In [2]:
from z3 import *

# Function to read the .dat file and return the parameters
def read_instance(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Parse m (number of couriers) and n (number of items)
    m = int(lines[0].strip())
    n = int(lines[1].strip())

    # Parse load limits
    load_limits = list(map(int, lines[2].strip().split()))

    # Parse item sizes
    item_sizes = list(map(int, lines[3].strip().split()))

    # Parse the distance matrix
    distance_matrix = []
    for i in range(n + 1):  # n+1 because the last row is the origin
        distance_matrix.append(list(map(int, lines[4 + i].strip().split())))

    return m, n, load_limits, item_sizes, distance_matrix

# Function to solve the MCP problem using Z3 Optimize
def solve_mcp(file_path):
    # Read instance data from .dat file
    m, n, load_limits, item_sizes, distance_matrix = read_instance(file_path)

    # Z3 optimizer initialization
    optimizer = Optimize()


    # Boolean variables: x[i][j] = True if courier i is assigned item j
    x = [[Bool(f"x_{i}_{j}") for j in range(n)] for i in range(m)]

    # Variables for the total distance traveled by each courier
    total_distance = [Int(f"D_{i}") for i in range(m)]

    # Maximum distance traveled by any courier (to minimize)
    max_distance = Int('Z')

    # 1. Each item must be assigned to exactly one courier
    for j in range(n):
        optimizer.add(Sum([If(x[i][j], 1, 0) for i in range(m)]) == 1)

    # 2. The load carried by each courier must not exceed its capacity
    for i in range(m):
        optimizer.add(Sum([If(x[i][j], item_sizes[j], 0) for j in range(n)]) <= load_limits[i])

    # 3. Distance calculation for each courier based on item assignments
    for i in range(m):
        # Sequential distances for assigned items
        courier_distance = []
        
        # From origin to the first item assigned and back to origin
        origin_distances = [If(x[i][j], distance_matrix[n][j] + distance_matrix[j][n], 0) for j in range(n)]
        
        # Sum the total travel distance
        optimizer.add(total_distance[i] == Sum(origin_distances))
        
    # 4. Objective: Minimize the maximum distance across all couriers
    for i in range(m):
        optimizer.add(max_distance >= total_distance[i])
    optimizer.minimize(max_distance)

    # Solve the problem
    if optimizer.check() == sat:
        model = optimizer.model()
        print(f"Minimized max distance: {model[max_distance]}")
        
        for i in range(m):
            assigned_items = [j for j in range(n) if model.evaluate(x[i][j])]
            print(f"Courier {i+1} is assigned items: {assigned_items} with distance {model[total_distance[i]]}")
    else:
        print("No solution found")




In [3]:
import os
import time
from concurrent.futures import ThreadPoolExecutor, TimeoutError

# Directory containing instance files
instance_folder = '/Users/shariqansari/Documents/CDMO/instances'

# Maximum allowed time for each instance (in seconds)
timeout_per_instance = 300

# Function to solve MCP problem for all files in the instance folder with a per-instance timeout
def solve_all_instances():
    # List and sort files in ascending order
    files = sorted([f for f in os.listdir(instance_folder) if f.endswith(".dat")])

    with ThreadPoolExecutor(max_workers=1) as executor:  # Using a single worker to keep order
        for filename in files:
            file_path = os.path.join(instance_folder, filename)
            print(f"Processing file: {file_path}")
            
            # Submit the solve_mcp task with a timeout
            future = executor.submit(solve_mcp, file_path)
            
            try:
                # Wait for the result with the specified timeout
                future.result(timeout=timeout_per_instance)
            except TimeoutError:
                print(f"Timeout: {file_path} took longer than {timeout_per_instance} seconds and was skipped.")
            except Exception as e:
                print(f"An error occurred while processing {file_path}: {e}")

# Call the function to start processing files
solve_all_instances()



Processing file: /Users/shariqansari/Documents/CDMO/instances/inst01.dat
Minimized max distance: 21
Courier 1 is assigned items: [2, 3, 5] with distance 21
Courier 2 is assigned items: [0, 1, 4] with distance 18
Processing file: /Users/shariqansari/Documents/CDMO/instances/inst02.dat
Minimized max distance: 306
Courier 1 is assigned items: [5] with distance 192
Courier 2 is assigned items: [3, 6] with distance 304
Courier 3 is assigned items: [4] with distance 192
Courier 4 is assigned items: [1, 2] with distance 290
Courier 5 is assigned items: [0, 8] with distance 306
Courier 6 is assigned items: [7] with distance 174
Processing file: /Users/shariqansari/Documents/CDMO/instances/inst03.dat
Minimized max distance: 20
Courier 1 is assigned items: [1, 3, 4] with distance 20
Courier 2 is assigned items: [2, 5] with distance 14
Courier 3 is assigned items: [0, 6] with distance 12
Processing file: /Users/shariqansari/Documents/CDMO/instances/inst04.dat
Minimized max distance: 220
Courier 1